In [1]:
import pandas as pd
import numpy as np
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter
import string
import re #regex library

In [2]:
data = pd.read_csv('datalabel.csv', error_bad_lines = False)
pd.set_option('max_colwidth', None)
data.head(10)

,Waktu,Data Tweet,Sentiment
0,9/5/2021 0:31,"Selamat pagi , bisa saja mereka minum kopi , Mereka dengan sengaja memanfaatkan aplikasi peduli lindungi",neutral
1,9/17/2021 12:48,Sosialisasi aplikasi Peduli Lindungi SATLANTAS POLRES PESISIR SELATAN #tmc polda metro,neutral
2,9/18/2021 1:44,"saya kan kemarin mendaftarkan orang tua saya vaksin di Peduli Lindungi, sekarang status nya seperti ini, terus langkah selanjutnya harus gimana ya?",neutral
3,9/22/2021 5:43,aplikasi peduli lindungi soalnya ada kata melindungimu,positive
4,9/17/2021 2:27,"Data NIK Tidak ditemukan di Peduli Lindungi, Hubungi Satu Data Vaksinasi di Pusat Telepon: Hot line Peduli Lindungi",negative
5,9/16/2021 5:23,Berantakannya aplikasi peduli lindungi,negative
6,9/9/2021 7:23,ini saya kalau mau masuk pakai scan barcode aplikasi peduli lindungi pasti begini terus. ada yang tahu kenapa?,neutral
7,9/4/2021 9:55,Intinya sebenarnya mereka semua ASN harus punya aplikasi peduli lindungi. Nanti kita Mintakan ke Kementrian Kesehatan QR code- nya,positive
8,9/21/2021 11:37,kawan kalau ke mall masih harus pakai aplikasi peduli lindungi?,neutral
9,9/21/2021 22:01,Nanti masuk UNPAD harus scan Peduli Lindungi tidak?,neutral


In [3]:
len(data)

3190

In [4]:
data[data.isnull().values.any(axis = 1)]

,Waktu,Data Tweet,Sentiment


In [ ]:
data = data[data['tweet'].notna()]

In [5]:
data[data.duplicated()]

,Waktu,Data Tweet,Sentiment


In [ ]:
data.drop_duplicates(keep=False')

In [ ]:
data[data.duplicated(keep='last')]

In [ ]:
len(data)

## CASE FOLDING

In [6]:
data['Data Tweet'] = data['Data Tweet'].str.lower()
data.head(10)

,Waktu,Data Tweet,Sentiment
0,9/5/2021 0:31,"selamat pagi , bisa saja mereka minum kopi , mereka dengan sengaja memanfaatkan aplikasi peduli lindungi",neutral
1,9/17/2021 12:48,sosialisasi aplikasi peduli lindungi satlantas polres pesisir selatan #tmc polda metro,neutral
2,9/18/2021 1:44,"saya kan kemarin mendaftarkan orang tua saya vaksin di peduli lindungi, sekarang status nya seperti ini, terus langkah selanjutnya harus gimana ya?",neutral
3,9/22/2021 5:43,aplikasi peduli lindungi soalnya ada kata melindungimu,positive
4,9/17/2021 2:27,"data nik tidak ditemukan di peduli lindungi, hubungi satu data vaksinasi di pusat telepon: hot line peduli lindungi",negative
5,9/16/2021 5:23,berantakannya aplikasi peduli lindungi,negative
6,9/9/2021 7:23,ini saya kalau mau masuk pakai scan barcode aplikasi peduli lindungi pasti begini terus. ada yang tahu kenapa?,neutral
7,9/4/2021 9:55,intinya sebenarnya mereka semua asn harus punya aplikasi peduli lindungi. nanti kita mintakan ke kementrian kesehatan qr code- nya,positive
8,9/21/2021 11:37,kawan kalau ke mall masih harus pakai aplikasi peduli lindungi?,neutral
9,9/21/2021 22:01,nanti masuk unpad harus scan peduli lindungi tidak?,neutral


## REMOVE PUNCTUATION

In [7]:
#pembersihan teks dari tab, new line, back slice, mention, link,hastag, URL
def remove_ulasan_special(text):
 #menghapus tab, new line, dan back slice
 text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
 #menghapus non ASCII (emoticon, chinese word, .etc)
 text = text.encode('ascii', 'replace').decode('ascii')
 #menghapus mention, link, hashtag
 text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ",text).split())
 #menghapus incomplete URL
 return text.replace("http://", " ").replace("https://", " ")
data['Data Tweet'] = data['Data Tweet'].apply(remove_ulasan_special)
data.head(10)

,Waktu,Data Tweet,Sentiment
0,9/5/2021 0:31,"selamat pagi , bisa saja mereka minum kopi , mereka dengan sengaja memanfaatkan aplikasi peduli lindungi",neutral
1,9/17/2021 12:48,sosialisasi aplikasi peduli lindungi satlantas polres pesisir selatan polda metro,neutral
2,9/18/2021 1:44,"saya kan kemarin mendaftarkan orang tua saya vaksin di peduli lindungi, sekarang status nya seperti ini, terus langkah selanjutnya harus gimana ya?",neutral
3,9/22/2021 5:43,aplikasi peduli lindungi soalnya ada kata melindungimu,positive
4,9/17/2021 2:27,"data nik tidak ditemukan di peduli lindungi, hubungi satu data vaksinasi di pusat telepon: hot line peduli lindungi",negative
5,9/16/2021 5:23,berantakannya aplikasi peduli lindungi,negative
6,9/9/2021 7:23,ini saya kalau mau masuk pakai scan barcode aplikasi peduli lindungi pasti begini terus. ada yang tahu kenapa?,neutral
7,9/4/2021 9:55,intinya sebenarnya mereka semua asn harus punya aplikasi peduli lindungi. nanti kita mintakan ke kementrian kesehatan qr code- nya,positive
8,9/21/2021 11:37,kawan kalau ke mall masih harus pakai aplikasi peduli lindungi?,neutral
9,9/21/2021 22:01,nanti masuk unpad harus scan peduli lindungi tidak?,neutral


In [8]:
# menghapus angka
def remove_number(text):
    return re.sub(r"\d+", "", text)
data['Data Tweet'] = data['Data Tweet'].apply(remove_number)
data.head(10)

,Waktu,Data Tweet,Sentiment
0,9/5/2021 0:31,"selamat pagi , bisa saja mereka minum kopi , mereka dengan sengaja memanfaatkan aplikasi peduli lindungi",neutral
1,9/17/2021 12:48,sosialisasi aplikasi peduli lindungi satlantas polres pesisir selatan polda metro,neutral
2,9/18/2021 1:44,"saya kan kemarin mendaftarkan orang tua saya vaksin di peduli lindungi, sekarang status nya seperti ini, terus langkah selanjutnya harus gimana ya?",neutral
3,9/22/2021 5:43,aplikasi peduli lindungi soalnya ada kata melindungimu,positive
4,9/17/2021 2:27,"data nik tidak ditemukan di peduli lindungi, hubungi satu data vaksinasi di pusat telepon: hot line peduli lindungi",negative
5,9/16/2021 5:23,berantakannya aplikasi peduli lindungi,negative
6,9/9/2021 7:23,ini saya kalau mau masuk pakai scan barcode aplikasi peduli lindungi pasti begini terus. ada yang tahu kenapa?,neutral
7,9/4/2021 9:55,intinya sebenarnya mereka semua asn harus punya aplikasi peduli lindungi. nanti kita mintakan ke kementrian kesehatan qr code- nya,positive
8,9/21/2021 11:37,kawan kalau ke mall masih harus pakai aplikasi peduli lindungi?,neutral
9,9/21/2021 22:01,nanti masuk unpad harus scan peduli lindungi tidak?,neutral


In [9]:
#menghapus tanda baca
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))
data['Data Tweet'] = data['Data Tweet'].apply(remove_punctuation)
data.head(10)

,Waktu,Data Tweet,Sentiment
0,9/5/2021 0:31,selamat pagi bisa saja mereka minum kopi mereka dengan sengaja memanfaatkan aplikasi peduli lindungi,neutral
1,9/17/2021 12:48,sosialisasi aplikasi peduli lindungi satlantas polres pesisir selatan polda metro,neutral
2,9/18/2021 1:44,saya kan kemarin mendaftarkan orang tua saya vaksin di peduli lindungi sekarang status nya seperti ini terus langkah selanjutnya harus gimana ya,neutral
3,9/22/2021 5:43,aplikasi peduli lindungi soalnya ada kata melindungimu,positive
4,9/17/2021 2:27,data nik tidak ditemukan di peduli lindungi hubungi satu data vaksinasi di pusat telepon hot line peduli lindungi,negative
5,9/16/2021 5:23,berantakannya aplikasi peduli lindungi,negative
6,9/9/2021 7:23,ini saya kalau mau masuk pakai scan barcode aplikasi peduli lindungi pasti begini terus ada yang tahu kenapa,neutral
7,9/4/2021 9:55,intinya sebenarnya mereka semua asn harus punya aplikasi peduli lindungi nanti kita mintakan ke kementrian kesehatan qr code nya,positive
8,9/21/2021 11:37,kawan kalau ke mall masih harus pakai aplikasi peduli lindungi,neutral
9,9/21/2021 22:01,nanti masuk unpad harus scan peduli lindungi tidak,neutral


In [10]:
#menghapus huruf tunggal (tidak berbentuk kata)
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)
data['Data Tweet'] = data['Data Tweet'].apply(remove_singl_char)
data.head(10)

,Waktu,Data Tweet,Sentiment
0,9/5/2021 0:31,selamat pagi bisa saja mereka minum kopi mereka dengan sengaja memanfaatkan aplikasi peduli lindungi,neutral
1,9/17/2021 12:48,sosialisasi aplikasi peduli lindungi satlantas polres pesisir selatan polda metro,neutral
2,9/18/2021 1:44,saya kan kemarin mendaftarkan orang tua saya vaksin di peduli lindungi sekarang status nya seperti ini terus langkah selanjutnya harus gimana ya,neutral
3,9/22/2021 5:43,aplikasi peduli lindungi soalnya ada kata melindungimu,positive
4,9/17/2021 2:27,data nik tidak ditemukan di peduli lindungi hubungi satu data vaksinasi di pusat telepon hot line peduli lindungi,negative
5,9/16/2021 5:23,berantakannya aplikasi peduli lindungi,negative
6,9/9/2021 7:23,ini saya kalau mau masuk pakai scan barcode aplikasi peduli lindungi pasti begini terus ada yang tahu kenapa,neutral
7,9/4/2021 9:55,intinya sebenarnya mereka semua asn harus punya aplikasi peduli lindungi nanti kita mintakan ke kementrian kesehatan qr code nya,positive
8,9/21/2021 11:37,kawan kalau ke mall masih harus pakai aplikasi peduli lindungi,neutral
9,9/21/2021 22:01,nanti masuk unpad harus scan peduli lindungi tidak,neutral


In [11]:
len(data)

3190

## TOKENISASI

In [12]:
def word_tokenize_wrapper(text):
    return word_tokenize(text)
data['Data Tweet'] = data['Data Tweet'].apply(word_tokenize_wrapper)
data.head(10)

,Waktu,Data Tweet,Sentiment
0,9/5/2021 0:31,"[selamat, pagi, bisa, saja, mereka, minum, kopi, mereka, dengan, sengaja, memanfaatkan, aplikasi, peduli, lindungi]",neutral
1,9/17/2021 12:48,"[sosialisasi, aplikasi, peduli, lindungi, satlantas, polres, pesisir, selatan, polda, metro]",neutral
2,9/18/2021 1:44,"[saya, kan, kemarin, mendaftarkan, orang, tua, saya, vaksin, di, peduli, lindungi, sekarang, status, nya, seperti, ini, terus, langkah, selanjutnya, harus, gimana, ya]",neutral
3,9/22/2021 5:43,"[aplikasi, peduli, lindungi, soalnya, ada, kata, melindungimu]",positive
4,9/17/2021 2:27,"[data, nik, tidak, ditemukan, di, peduli, lindungi, hubungi, satu, data, vaksinasi, di, pusat, telepon, hot, line, peduli, lindungi]",negative
5,9/16/2021 5:23,"[berantakannya, aplikasi, peduli, lindungi]",negative
6,9/9/2021 7:23,"[ini, saya, kalau, mau, masuk, pakai, scan, barcode, aplikasi, peduli, lindungi, pasti, begini, terus, ada, yang, tahu, kenapa]",neutral
7,9/4/2021 9:55,"[intinya, sebenarnya, mereka, semua, asn, harus, punya, aplikasi, peduli, lindungi, nanti, kita, mintakan, ke, kementrian, kesehatan, qr, code, nya]",positive
8,9/21/2021 11:37,"[kawan, kalau, ke, mall, masih, harus, pakai, aplikasi, peduli, lindungi]",neutral
9,9/21/2021 22:01,"[nanti, masuk, unpad, harus, scan, peduli, lindungi, tidak]",neutral


In [13]:
data = data.drop(columns=['Waktu'])

In [14]:
data

,Data Tweet,Sentiment
0,"[selamat, pagi, bisa, saja, mereka, minum, kopi, mereka, dengan, sengaja, memanfaatkan, aplikasi, peduli, lindungi]",neutral
1,"[sosialisasi, aplikasi, peduli, lindungi, satlantas, polres, pesisir, selatan, polda, metro]",neutral
2,"[saya, kan, kemarin, mendaftarkan, orang, tua, saya, vaksin, di, peduli, lindungi, sekarang, status, nya, seperti, ini, terus, langkah, selanjutnya, harus, gimana, ya]",neutral
3,"[aplikasi, peduli, lindungi, soalnya, ada, kata, melindungimu]",positive
4,"[data, nik, tidak, ditemukan, di, peduli, lindungi, hubungi, satu, data, vaksinasi, di, pusat, telepon, hot, line, peduli, lindungi]",negative
...,...,...
3185,"[yuanita, menambahkan, peserta, vaksinasi, harus, mendaftar, di, elektronik, vaksin, yang, tersedia, di, website, isi, biodata, dan, mengisi, formulir, vaksin]",positive
3186,"[yuk, kita, pakai, peduli, lindungi]",positive
3187,"[yuk, segera, unduh, aplikasi, peduli, lindungi, mulai, tanggal, september, masuk, supermarket, kini, wajib, menggunakan, peduli, lidungi]",positive
3188,"[yuk, sobat, segera, pasang, atau, unduh, aplikasi, peduli, lindungi, ya, agar, kita, dapat, selalu, mencegah, penyebaran, covid]",positive


## STOPWORD REMOVAL

In [15]:
list_stopwords = stopwords.words('indonesian')
list_stopwords.extend(['uu','cipta','kerja'])

def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]
data['Data Tweet'] = data['Data Tweet'].apply(stopwords_removal)
data.head(10)

,Data Tweet,Sentiment
0,"[selamat, pagi, minum, kopi, sengaja, memanfaatkan, aplikasi, peduli, lindungi]",neutral
1,"[sosialisasi, aplikasi, peduli, lindungi, satlantas, polres, pesisir, selatan, polda, metro]",neutral
2,"[kemarin, mendaftarkan, orang, tua, vaksin, peduli, lindungi, status, nya, langkah, gimana, ya]",neutral
3,"[aplikasi, peduli, lindungi, melindungimu]",positive
4,"[data, nik, ditemukan, peduli, lindungi, hubungi, data, vaksinasi, pusat, telepon, hot, line, peduli, lindungi]",negative
5,"[berantakannya, aplikasi, peduli, lindungi]",negative
6,"[masuk, pakai, scan, barcode, aplikasi, peduli, lindungi]",neutral
7,"[intinya, asn, aplikasi, peduli, lindungi, mintakan, kementrian, kesehatan, qr, code, nya]",positive
8,"[kawan, mall, pakai, aplikasi, peduli, lindungi]",neutral
9,"[masuk, unpad, scan, peduli, lindungi]",neutral


## STEMMING

In [16]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in data['Data Tweet']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '
            
print(len(term_dict))
print("------------------------")

for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    print(term,":" ,term_dict[term])
    
print(term_dict)
print("------------------------")


# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

data['Data Tweet'] = data['Data Tweet'].swifter.apply(get_stemmed_term)
print(data['Data Tweet'])

3083
------------------------
selamat : selamat
pagi : pagi
minum : minum
kopi : kopi
sengaja : sengaja
memanfaatkan : manfaat
aplikasi : aplikasi
peduli : peduli
lindungi : lindung
sosialisasi : sosialisasi
satlantas : satlantas
polres : polres
pesisir : pesisir
selatan : selatan
polda : polda
metro : metro
kemarin : kemarin
mendaftarkan : daftar
orang : orang
tua : tua
vaksin : vaksin
status : status
nya : nya
langkah : langkah
gimana : gimana
ya : ya
melindungimu : lindung
data : data
nik : nik
ditemukan : temu
hubungi : hubung
vaksinasi : vaksinasi
pusat : pusat
telepon : telepon
hot : hot
line : line
berantakannya : beranta
masuk : masuk
pakai : pakai
scan : scan
barcode : barcode
intinya : inti
asn : asn
mintakan : minta
kementrian : tri
kesehatan : sehat
qr : qr
code : code
kawan : kawan
mall : mall
unpad : unpad
syarat : syarat
pemeriksaan : periksa
memasuki : pasuk
september : september
supermarket : supermarket
hypermart : hypermart
wajib : wajib
riwayat : riwayat
perjalanan 

online : online
jarak : jarak
tertumpuk : tumpuk
sorenya : sore
dibagian : bagi
titik : titik
pdf : pdf
mencetak : cetak
terintegrasi : integrasi
loh : loh
pengalaman : alam
lapor : lapor
disdukcapil : disdukcapil
tanggapan : tanggap
malam : malam
sahabat : sahabat
persyaratan : syarat
tergantung : gantung
bioskop : bioskop
senin : senin
barcodenya : barcodenya
rekam : rekam
jejak : jejak
geser : geser
warna : warna
petugas : tugas
pinggir : pinggir
oh : oh
pikir : pikir
teleponnya : telepon
input : input
ngurus : ngurus
kirim : kirim
yak : yak
bali : bal
bandung : bandung
bepergian : pergi
diterima : terima
perhatikan : perhati
mengurangi : kurang
aktifitas : aktifitas
rumah : rumah
pengalamanku : alam
diverifikasi : verifikasi
emai : emai
semalam : malam
rabu : rabu
kamis : kamis
minggu : minggu
diweb : diweb
scannya : scannya
sosial : sosial
media : media
menara : menara
bank : bank
mega : mega
jakarta : jakarta
statusnya : status
praktis : praktis
rdt : rdt
penebel : penebel
nih : 

himna : himna
ketahuilah : tahu
arti : arti
ketahui : tahu
kriteria : kriteria
komifonya : komifonya
sibuk : sibuk
membenahi : benah
lambat : lambat
busway : busway
pengaturan : atur
dipaksa : paksa
perawatnya : awat
dilaksanakan : laksana
perkantoran : kantor
fikir : fikir
mcdonals : mcdonals
kimia : kimia
farma : farma
ui : ui
ux : ux
pajak : pajak
indomaret : indomaret
parkiran : parkir
bebas : bebas
jualan : jual
disalin : salin
dibikin : bikin
ditelusuri : telusur
ketahuan : tahu
keluarnya : keluar
suami : suami
nakal : nakal
awasi : awas
landasan : landas
hukum : hukum
spesifik : spesifik
melanggar : langgar
pasal : pasal
terapkan : terap
enak : enak
korupsi : korupsi
koruptornya : koruptor
disaat : saat
pandemi : pandemi
miskin : miskin
bertahan : tahan
hidup : hidup
gerak : gerak
pemasaran : pasar
penjualan : jual
konsep : konsep
pejabat : jabat
masuklah : masuk
memaksakan : paksa
annisa : annisa
cs : cs
kampung : kampung
yogyakarta : yogyakarta
international : international
ai

bantuin : bantuin
formulirnya : formulir
koruptor : koruptor
berbagi : bagi
larinya : lari
ngatur : ngatur
pengisian : isi
salatiga : salatiga
hermina : hermina
barangkali : barangkali
rizqi : rizqi
subject : subject
teruntuk : untuk
pembatasan : batas
berjaga : jaga
mengulang : ulang
ciptakan : cipta
pastinya : pasti
beritahu : beritahu
jomblo : jomblo
persis : persis
fotonya : foto
facebook : facebook
twitter : twitter
keamanannya : aman
direspon : direspon
relasi : relasi
makassar : makassar
cafe : cafe
suntik : suntik
maksimal : maksimal
didaftarin : didaftarin
ragu : ragu
sepengalaman : alam
batuk : batuk
pilek : pilek
perbolehkan : boleh
tersimpan : simpan
saoalnya : saoalnya
ingatlah : ingat
guru : guru
mail : mail
memuat : muat
mewajibkan : wajib
masker : masker
ganda : ganda
dikeluarkan : keluar
dilakukannya : laku
an : an
kapasitas : kapasitas
vakin : vakin
kejadian : jadi
bandaranya : bandara
hasilnya : hasil
laboratorium : laboratorium
fisiknya : fisik
bpn : bpn
bolak : bol

kirain : kirain
fogood : fogood
menghapus : hapus
menunggu : tunggu
pengumuman : umum
mengunggah : unggah
tempo : tempo
membahas : bahas
bocornya : bocor
prediksi : prediksi
bintang : bintang
kurangin : rangin
kerjaan : kerja
pondok : pondok
pertanda : tanda
palu : palu
grand : grand
penjelasannya : jelas
bentar : bentar
seblak : seblak
beranjak : anjak
tidur : tidur
petangnya : petang
shang : shang
chi : chi
insyaallah : insyaallah
dana : dana
perkiraan : kira
investasi : investasi
pengembangan : kembang
mensyaratkan : syarat
berbasis : bas
diadopsi : adopsi
dimiliki : milik
republik : republik
berharap : harap
pantai : pantai
berhubung : hubung
gambarnya : gambar
suasana : suasana
cgv : cgv
transmart : transmart
pembukaannya : buka
gabungan : gabung
berusaha : usaha
aeon : aeon
sentul : sentul
belanjanya : belanja
bright : bright
istana : istana
sempurna : sempurna
tertib : tertib
lintas : lintas
batu : batu
si : si
hangat : hangat
sebatas : batas
kesebar : sebar
bodoh : bodoh
milih 

google : google
foursquare : foursquare
ikutin : ikutin
lansia : lansia
diablo : diablo
omongan : omong
kenyamanan : nyaman
berdagang : dagang
terjamin : jamin
anggota : anggota
wag : wag
kepakai : pakai
clubhouse : clubhouse
temanya : tema
kerensistem : kerensistem
manajemen : manajemen
ketawa : ketawa
ketiga : tiga
situasi : situasi
suratan : surat
memudahkan : mudah
tingkat : tingkat
klarifikasi : klarifikasi
selingkuh : selingkuh
hilanginnya : hilanginnya
notif : notif
diluncurkan : luncur
bersangkutan : sangkut
bagikan : bagi
panduan : pandu
menyangkal : sangkal
ditangkap : tangkap
bertambah : tambah
dedy : dedy
permadi : madi
komisaris : komisaris
langsungkan : langsung
pernikahan : nikah
putranya : putra
konferensi : konferensi
pers : pers
pemegang : pegang
nungguin : nungguin
artikel : artikel
membedah : bedah
aktor : aktor
dibalik : balik
laku : laku
politisi : politis
sukanya : suka
kawannya : kawan
maksimalkan : maksimal
pemanfaatan : manfaat
pekerja : kerja
fansly : fansly


sentani : sentani
jayapura : jayapura
kontingen : kontingen
stress : stress
lengang : lengang
beroperasinya : operasi
movimax : movimax
jatim : jatim
datengin : datengin
tulang : tulang
tanggapin : tanggapin
traveling : traveling
skrining : skrining
pon : pon
xx : xx
mencegah : cegah
timbul : timbul
klust : klust
store : store
berlokasi : lokasi
diberlaku : laku
halte : halte
mess : mess
laporannya : lapor
dibolak : bolak
balikin : balikin
mencuri : curi
transpotasi : transpotasi
peran : peran
peduliin : peduliin
jambi : jambi
muhasabah : muhasabah
balasnya : balas
akomodasi : akomodasi
film : film
pedulli : pedulli
dibahas : bahas
senjata : senjata
laju : laju
pelabuhan : labuh
gilimanuk : gilimanuk
ahli : ahli
devie : devie
rahmawati : rahmawati
ringan : ringan
terlepas : lepas
chip : chip
terjawab : jawab
terpantau : pantau
dibohongi : bohong
dijawablah : jawab
mencela : cela
bersabar : sabar
penyalahgunaan : penyalahgunaan
spesialis : spesialis
arena : arena
radius : radius
ngomong

Pandas Apply:   0%|          | 0/3190 [00:00<?, ?it/s]

0                                          [selamat, pagi, minum, kopi, sengaja, manfaat, aplikasi, peduli, lindung]
1                        [sosialisasi, aplikasi, peduli, lindung, satlantas, polres, pesisir, selatan, polda, metro]
2                           [kemarin, daftar, orang, tua, vaksin, peduli, lindung, status, nya, langkah, gimana, ya]
3                                                                               [aplikasi, peduli, lindung, lindung]
4            [data, nik, temu, peduli, lindung, hubung, data, vaksinasi, pusat, telepon, hot, line, peduli, lindung]
                                                            ...                                                     
3185    [yuanita, serta, vaksinasi, daftar, elektronik, vaksin, sedia, website, isi, biodata, isi, formulir, vaksin]
3186                                                                                   [yuk, pakai, peduli, lindung]
3187         [yuk, unduh, aplikasi, peduli, lindung, tanggal, se

In [ ]:
data

In [17]:
data.to_csv(r'D:\Dummy.csv', index = False)